In [3]:
import torch

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU available")

Tesla V100S-PCIE-32GB


In [5]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 10.5 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105789 sha256=edbd5823885c5552e312b9bd46d807ba09aa243bca7d8c49f21e2a0c713edef8
  Stored in directory: /mnt/ufs18/home-086/madired3/.cache/pip/wheels/ff/55/fb/b27a466be754d2a06ffe0e37b248d844f090a63b51becea85d
Successfully built kaggle

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
print(len(os.listdir("./dataset_20210629145407_top_600/annotations")))
print(len(os.listdir("./dataset_20210629145407_top_600/images")))

167802
37265


In [3]:
# remove images and labels of index > 30000
i=0
for i, label_file in enumerate(sorted(os.listdir("./dataset_20210629145407_top_600/annotations"))):
    if i>30000:
        src_image_path = os.path.join("./dataset_20210629145407_top_600/annotations", label_file)
        os.remove(src_image_path)

for i, image_file in enumerate(sorted(os.listdir("./dataset_20210629145407_top_600/images"))):
    if i>30000:
        src_image_path = os.path.join("./dataset_20210629145407_top_600/images", image_file)
        os.remove(src_image_path)


In [1]:
!kaggle datasets download -d dreamfactor/biggest-lego-dataset-600-parts

Dataset URL: https://www.kaggle.com/datasets/dreamfactor/biggest-lego-dataset-600-parts
License(s): CC0-1.0
100%|█████████████████████████████████████▉| 7.68G/7.68G [01:37<00:00, 83.6MB/s]
100%|██████████████████████████████████████| 7.68G/7.68G [01:37<00:00, 84.9MB/s]


In [ ]:
!unzip -q biggest-lego-dataset-600-parts.zip

dataset_20210629145407_top_600/images/39037eb4-d9df-11eb-a530-3497f683a169.jpg:  write error (disk full?).  Continue? (y/n/^C) 

In [3]:
import xml.etree.ElementTree as ET
import os

def pascal_voc_to_yolo(annotations_dir, image_dir, output_dir):
    """
    Converts PASCAL VOC format annotations to YOLO format.

    Args:
        annotations_dir: Path to the directory containing PASCAL VOC XML files.
        image_dir: Path to the directory containing the corresponding images.
        output_dir: Path to the directory to save the YOLO text files.
    """

    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

    category_map = {} # Dictionary to store category names and their indices
    category_id = 0


    for filename in os.listdir(annotations_dir):
        if filename.endswith(".xml"):
            annotation_path = os.path.join(annotations_dir, filename)
            try:
                tree = ET.parse(annotation_path)
                root = tree.getroot()

                image_name = root.find("filename").text
                image_path = os.path.join(image_dir, image_name)  # Construct image path
                width = int(root.find("size").find("width").text)
                height = int(root.find("size").find("height").text)


                yolo_annotations = []

                for obj in root.findall("object"):
                    category_name = obj.find("name").text
                        # category_id += 1

                    bbox = obj.find("bndbox")
                    xmin = int(bbox.find("xmin").text)
                    ymin = int(bbox.find("ymin").text)
                    xmax = int(bbox.find("xmax").text)
                    ymax = int(bbox.find("ymax").text)


                    x_center = (xmin + xmax) / (2 * width)
                    y_center = (ymin + ymax) / (2 * height)
                    w_norm = (xmax - xmin) / width
                    h_norm = (ymax - ymin) / height

                    yolo_annotation = f"0 {x_center} {y_center} {w_norm} {h_norm}\n"
                    yolo_annotations.append(yolo_annotation)


                output_filename = filename[:-4] + ".txt"  # Replace .xml with .txt
                output_path = os.path.join(output_dir, output_filename)

                with open(output_path, "w") as f:
                    f.writelines(yolo_annotations)

            except ET.ParseError as e:
                print(f"Error parsing {annotation_path}: {e}")
            except FileNotFoundError:
                print(f"Image file not found: {image_path}")  # Handle missing images


    # Create a categories.txt file (optional but useful)
    categories_path = os.path.join(output_dir, "categories.txt")
    with open(categories_path, "w") as f:
        for category_name, category_id in category_map.items():
            f.write(f"{category_name}\n")

# Example Usage
annotations_dir = "./dataset_20210629145407_top_600/annotations/"
image_dir = "./dataset_20210629145407_top_600/images/"  # Path to the directory with images
output_dir = "./dataset_20210629145407_top_600/labels/"
pascal_voc_to_yolo(annotations_dir, image_dir, output_dir)

Error parsing ./dataset_20210629145407_top_600/annotations/225122c6-db87-11eb-bf78-3497f683a169.xml: not well-formed (invalid token): line 1, column 0


In [6]:
import os
import shutil

# Define your directories
annotations_dir = "./dataset_20210629145407_top_600/annotations/"
image_dir = "./dataset_20210629145407_top_600/images/"
output_dir = "./dataset_20210629145407_top_600/labels/"

# Define train and validation directories
train_image_dir = "./datasets/images/train/"
val_image_dir = "./datasets/images/val/"
train_label_dir = "./datasets/labels/train/"
val_label_dir = "./datasets/labels/val/"

# Create train/val directories if they don't exist
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Get a sorted list of image and label files
image_files = sorted(os.listdir(image_dir))
label_files = sorted(os.listdir(output_dir))

# Move images to train and val folders
for i, image_file in enumerate(image_files):
    src_image_path = os.path.join(image_dir, image_file)
    if i < 2000:
        dest_image_path = os.path.join(val_image_dir, image_file)
        shutil.move(src_image_path, dest_image_path)
    elif i<=22000:
        dest_image_path = os.path.join(train_image_dir, image_file)
        shutil.move(src_image_path, dest_image_path)
    else:
        os.remove(src_image_path)

    
# Move labels to train and val folders
for i, label_file in enumerate(label_files):
    src_label_path = os.path.join(output_dir, label_file)
    if i < 2000:
        dest_label_path = os.path.join(val_label_dir, label_file)
        shutil.move(src_label_path, dest_label_path)    
    elif i<=22000:
        dest_label_path = os.path.join(train_label_dir, label_file)
        shutil.move(src_label_path, dest_label_path)
    else:
      os.remove(src_image_path)
    

print("Files have been successfully split into train and val folders.")


FileNotFoundError: [Errno 2] No such file or directory: './dataset_20210629145407_top_600/images/ffffefc2-da91-11eb-8c79-3497f683a169.jpg'

In [7]:
print(len(label_files))
print(len(image_files))

167802
167800


In [9]:
# %pip install ultralytics
%pip install -U ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.31 🚀 Python-3.11.3 torch-2.5.1+cu124 CUDA:0 (Tesla V100S-PCIE-32GB, 32494MiB)
Setup complete ✅ (48 CPUs, 187.3 GB RAM, 12.8/25.4 GB disk)


In [3]:
from ultralytics import YOLO

# Load the YOLOv11 model
model = YOLO('./yolo11l.pt')  # Replace with the path to yolov11.pt

# Train the model
model.train(data='./lego.yaml', epochs=20, batch=-1, imgsz=640)

# Save the trained model
trained_model_path = 'yolo11x_trained.pt'
model.save(trained_model_path)

print(f"Model saved as: {trained_model_path}")


Ultralytics 8.3.32 🚀 Python-3.11.3 torch-2.5.1+cu124 CUDA:0 (Tesla V100S-PCIE-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=./yolo11l.pt, data=./lego.yaml, epochs=20, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning /mnt/ufs18/home-086/madired3/cse841/datasets/labels/train.cache... 20000 images, 1 backgrounds, 0 corrupt: 100%|██████████| 20001/20001 [00:00<?, ?it/s]


AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla V100S-PCIE-32GB) 31.73G total, 0.24G reserved, 0.22G allocated, 31.27G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    25311251       87.27         0.960         69.96         151.2        (1, 3, 640, 640)                    list
    25311251       174.5         1.625         59.57         78.83        (2, 3, 640, 640)                    list
    25311251       349.1         2.898         273.8         78.28        (4, 3, 640, 640)                    list
    25311251       698.2         5.450         70.39         86.54        (8, 3, 640, 640)                    list
    25311251        1396        10.597         84.96         116.2       (16, 3, 640, 640)                    list
    25311251        2793        20.321         168.8           194       (32, 3, 640, 640)                    list
CUDA o

train: Scanning /mnt/ufs18/home-086/madired3/cse841/datasets/labels/train.cache... 20000 images, 1 backgrounds, 0 corrupt: 100%|██████████| 20001/20001 [00:00<?, ?it/s]
val: Scanning /mnt/ufs18/home-086/madired3/cse841/datasets/labels/val.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]


Plotting labels to runs/detect/train15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.000453125), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train15
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      18.9G     0.2996     0.2106     0.8921         56        640: 100%|██████████| 690/690 [05:19<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:14<00:00,  2.50it/s]

                   all       2000       6566      0.983      0.977      0.994      0.971



20 epochs completed in 1.855 hours.
Optimizer stripped from runs/detect/train15/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/train15/weights/best.pt, 51.2MB

Validating runs/detect/train15/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.11.3 torch-2.5.1+cu124 CUDA:0 (Tesla V100S-PCIE-32GB, 32494MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:16<00:00,  2.17it/s]


                   all       2000       6566      0.983      0.977      0.994      0.971
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train15
Model saved as: yolo11x_trained.pt


In [ ]:
from ultralytics import YOLO  # Import YOLO framework

# Load the YOLOv11 model
model = YOLO("yolo11x_trained.pt")  # Path to your trained model

# Perform inference on the image
results = model("./dataset_20210629145407_top_600/JPEGImages/0a0abab6-daf8-11eb-a7fc-3497f683a169.jpg")  # Path to the test image

# Print detections
print("Detections: ",len(results[0].boxes))

for result in results:
    print(result)  # Adjust this to inspect bounding boxes, class scores, etc.



image 1/1 /mnt/ufs18/home-086/madired3/cse841/000a2e34-d9d5-11eb-b337-3497f683a169.jpg: 640x640 4 legos, 23.2ms
Speed: 3.1ms preprocess, 23.2ms inference, 11.5ms postprocess per image at shape (1, 3, 640, 640)
Detections:  4
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'lego'}
obb: None
orig_img: array([[[ 21,  48,  82],
        [ 21,  48,  82],
        [ 21,  48,  82],
        ...,
        [ 83, 130, 174],
        [ 84, 131, 175],
        [ 84, 131, 175]],

       [[ 22,  49,  83],
        [ 23,  50,  84],
        [ 25,  52,  86],
        ...,
        [ 77, 124, 168],
        [ 79, 126, 170],
        [ 78, 127, 171]],

       [[ 26,  53,  87],
        [ 28,  55,  89],
        [ 30,  57,  91],
        ...,
        [ 69, 118, 162],
        [ 71, 120, 164],
        [ 71, 122, 165]],

       ...,

       [[ 39,  68,  99],
        [ 39,  68,  99],
        [ 38,  67,  98],
        ...,
    

In [12]:
print(result[0].boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9719], device='cuda:0')
data: tensor([[182.2321, 363.4385, 421.9596, 480.0490,   0.9719,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (600, 600)
shape: torch.Size([1, 6])
xywh: tensor([[302.0959, 421.7437, 239.7275, 116.6105]], device='cuda:0')
xywhn: tensor([[0.5035, 0.7029, 0.3995, 0.1944]], device='cuda:0')
xyxy: tensor([[182.2321, 363.4385, 421.9596, 480.0490]], device='cuda:0')
xyxyn: tensor([[0.3037, 0.6057, 0.7033, 0.8001]], device='cuda:0')


In [1]:
%reset -f
import gc
gc.collect()

0

In [1]:
pip install jupyter

Defaulting to user installation because normal site-packages is not writeable
  Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!nvidia-smi

Thu Nov 28 17:49:08 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          Off |   00000000:3D:00.0 Off |                  Off |
| N/A   38C    P0             54W /  250W |   20572MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [21]:
results = model.val(data='./lego.yaml', imgsz=640)
print("Validation results:")
print(results)

Ultralytics 8.3.31 🚀 Python-3.11.3 torch-2.5.1+cu124 CUDA:0 (Tesla V100S-PCIE-32GB, 32494MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /mnt/ufs18/home-086/madired3/cse841/datasets/labels/val.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.18it/s]


                   all        100        330      0.993      0.955       0.99      0.963
Speed: 0.3ms preprocess, 12.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train22
Validation results:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x14eec5d75850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,

In [18]:
%pip install google-colab


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)
ERROR: No matching distribution found for google-colab

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
